# 第11章 条件随机场


1．概率无向图模型是由无向图表示的联合概率分布。无向图上的结点之间的连接关系表示了联合分布的随机变量集合之间的**条件独立性**，即马尔可夫性。因此，概率无向图模型也称为马尔可夫随机场。

概率无向图模型或马尔可夫随机场的联合概率分布可以分解为无向图最大团上的正值函数的乘积的形式。

2．条件随机场是给定输入随机变量$X$条件下，输出随机变量$Y$的条件概率分布模型， 其形式为参数化的对数线性模型。条件随机场的最大特点是假设输出变量之间的联合概率分布构成概率无向图模型，即马尔可夫随机场。条件随机场是判别模型。

3．线性链条件随机场是定义在观测序列与标记序列上的条件随机场。线性链条件随机场一般表示为给定观测序列条件下的标记序列的条件概率分布，由参数化的对数线性模型表示。模型包含特征及相应的权值，特征是定义在线性链的边与结点上的。线性链条件随机场的数学表达式是
$$
P(y | x)=\frac{1}{Z(x)} \exp \left(\sum_{i, k} \lambda_{k} t_{k}\left(y_{i-1}, y_{i}, x, i\right)+\sum_{i, l} \mu_{l} s_{l}\left(y_{i}, x, i\right)\right)
$$

其中，
 $$
Z(x)=\sum_{y} \exp \left(\sum_{i, k} \lambda_{k} t_{k}\left(y_{i-1}, y_{i}, x, i\right)+\sum_{i, l} \mu_{l} s_{l}\left(y_{i}, x, i\right)\right)
$$

4．线性链条件随机场的概率计算通常利用前向-后向算法。

5．条件随机场的学习方法通常是极大似然估计方法或正则化的极大似然估计，即在给定训练数据下，通过极大化训练数据的对数似然函数以估计模型参数。具体的算法有改进的迭代尺度算法、梯度下降法、拟牛顿法等。

6．线性链条件随机场的一个重要应用是标注。维特比算法是给定观测序列求条件概率最大的标记序列的方法。


## 概率无向图模型
概率无向图模型又称**马尔可夫随机场(Markov Random Field, MRF)**，是一个可以由**满足以下三个性质的无向图**表示的**联合概率分布**。

- 成对马尔可夫性  
  给定随机变量组$Y_O$的条件下随机变量$Y_u$和$Y_v$是条件独立的  
  在无向图中$u和$都是没有连接边的结点  
  $P(Y_u,Y_v|Y_O)=P(Y_u|Y_O)P(Y_v|Y_O)$

- 局部马尔可夫性  
  给定随机变量组$Y_W​$的条件下随机变量$Y_v​$与随机变量组$Y_O​$是独立的  
  在无向图中$W$是所有与$v$有连接边的结点集合, $O$是$v$和$W$以外的其他所有节点  
  $P(Y_v,Y_O|Y_W)=P(Y_v|Y_W)P(Y_O|Y_W)​$

- 全局马尔可夫性  
  给定随机变量组$Y_C$的条件下随机变量组$Y_A$和$Y_B$是条件独立的  
  在无向图中$A$和$B$是被结点集合$C$分开的任意结点集合  
  $P(Y_A,Y_B|Y_C)=P(Y_A|Y_C)P(Y_B|Y_C)$


### MRF因子分解
事实上, 我们更关心如何根据概率无向图模型求联合概率分布.对给定的概率无向图模型, 我们希望将整体的联合概率写成若干子联合概率的乘积的形式, 也就是将联合概率进行因子分解.
#### 团与最大团
无向图$G$中任何两个结点均有边连接的节点子集称为**团**(clique), 若一个团$C$不能通过添加任何一个$G$中的其他节点的方式使其变得更大, 那么称$C$是**最大团**(maximal clique)
#### 因子分解
用$Y_C$来表示$C$对应的随机变量. 概率无向图模型的联合分布$P(Y)$可以写作图中所有最大团$C$上的函数$\Psi_C(Y_C)$的乘积的形式, 即
$$P(Y) = \frac 1 Z \prod_C \Psi_C(Y_C)$$ 
其中, $Z$是规范化因子,
$$Z = \sum_Y\prod_C \Psi_C(Y_C)$$
函数$\Psi_C(Y_C)$称为$势函数$.这里要求势函数要严格正, 通常定义为指数函数
$$\Psi_C(Y_C) = exp\{-E(Y_C)\}$$

## 条件随机场

条件随机场是给定随机变量$X$条件下，随机变量$Y$的马尔可夫随机场。
设$X$与$Y$是随机变量, $P(Y|X)$是在给定$X$的条件下$Y$的概率分布.若随机变量$Y$构成一个由无向图$G=(V, E)$表示的马尔可夫随机场, 即
$$P(Y_v|X, Y_w, w \neq v)= P(Y_v|X, Y_w, w\sim v)$$
对任意结点$v$成立, 则称条件概率分布$P(Y|X)$为条件随机场. 式中$w \sim v$表示图$G$中与结点$v$相连的结点$w$, $w\neq v$表示结点$v$以为的所有结点.

一般假设$X和Y$具有相同的图结构.即
$$G = (V=\{1, 2, \cdots, n\}, E=\{(i, i+1)\}), \quad i=1, 2, \cdots, n-1$$
在此情况下, $X=(X_1, X_2, \cdots, X_n), Y=(Y_1, Y_2, \cdots, Y_n)$, 最大团是相邻的两个结点.
### 线性链条件随机场

设$X=(X_1,X_2,\cdots,X_n)，Y=(Y_1,Y_2,\cdots,Y_n)$均为线性链表示的随机变量序列，若在给定随机变量序列$X$的条件下，随机变量序列$Y$的条件概率分布$P(Y|X)$构成条件随机场，即满足马尔可夫性
$$
P(Y_i|X,Y_1,\cdots,Y_{i-1},Y_{i+1},\cdots,Y_n)=P(Y_i|X,Y_{i-1},Y_{i+1})\\
i=1,2,\cdots,n (在i=1和n时只考虑单边)
$$
则称$P(Y|X)$为线性链条件随机场。**在标注问题中**，$X$表示输入观测序列， $Y$表示输出标记序列或状态序列。
### 特征函数

线性链条件随机场的参数化形式
$$
P(y|x)=\frac{1}{Z(x)}\exp\left(\sum\limits_{i,k}\lambda_kt_k(y_{i-1},y_i,x,i)+\sum_{i,l}\mu_ls_l(y_i,x,i)\right)
$$
其中

$t_k$是定义在**边**上的特征函数，称为转移特征, 依赖于当前和前一个位置

$s_l$是定义在**结点**上的特征函数，称为状态特征, 依赖于当前位置

注意到这种表达就是不同特征的加权求和形式，$t_k,s_l$都依赖于位置，是局部特征函数。通常特征函数$t_k和s_l$取值为0或1: 当满足特征条件时取1, 否则取0.

$k,l$对应特征函数的编号，注意这里用了$k,l$两个编号，$i$对应了输出序列的每个位置
### 对数线性模型

线性链条件随机场也是**对数线性模型**(定义在时序数据上的)。

条件随机场可以看做是最大熵马尔可夫模型在标注问题上的推广。


### 例 11.1

In [21]:
import numpy as np
#这里定义T为转移矩阵列代表前一个y(ij)代表由状态i转到状态j的概率,Tx矩阵x对应于时间序列

#这里将书上的转移特征转换为如下以时间轴为区别的三个多维列表，维度为输出的维度
T1 = [[0.6, 1], [1, 0]]  # y_1 -> ｙ_2
T2 = [[0, 1], [1, 0.2]]  # y_2 -> y_3
 
#将书上的结点状态特征 未规范化概率
S0 = [1, 0.5]  # y1  y1=1, w=1; y1=2, w=0.5
S1 = [0.8, 0.5]
S2 = [0.8, 0.5]

Y = [1, 2, 2]  #即书上例一需要计算的非规范化条件概率的标记序列
Y = np.array(Y) - 1  #这里为了将数与索引相对应即从零开始
P = np.exp(S0[Y[0]])
for i in range(1, len(Y)):
    # exp(S_i[Y_i] + T_i[y_{i-1}][y_i])
    P *= np.exp((eval('S%d' % i)[Y[i]]) + eval('T%d' % i)[Y[i - 1]][Y[i]])
print(P)
print(np.exp(3.2))

24.532530197109345
24.532530197109352


### 简化形式
线性链条件随机场的参数化形式中同一特征在各个位置都有定义, 可以对同一个特征在各个位置求和, 将局部特征函数转为一个全局特征函数, 将条件随机场写成权值向量和特征向量的内积形式.

设有$K_1$个转移特征， $K_2$个状态特征, $K=K_1+K_2$

$$
\color{red}{
f_k(y_{i-1},y_i,x,i)=
\begin{cases}
t_k(y_{i-1},y_i,x,i),&k=1,2,\dots,K_1\\
s_l(y_i,x,i),&k=K_1+l;l=1,2,\dots,K_2
\end{cases}
}
$$
然后，对转和状态特征在各个位置$i$求和，记作
$$
f_k(y,x)=\sum_{i=1}^nf_k(y_{i-1},y_i,x,i),k=1,2,\dots,K
$$
用$w_k$表示特征$f_k(y,x)$的权值
$$
w_k=
\begin{cases}
\lambda_k,&k=1,2,\dots,K_1\\
\mu_l,&k=K1+l;l=1,2,\dots,K_2
\end{cases}
$$
于是条件随机场可以表示为
$$
\begin{align}
P(y|x)&=\frac{1}{Z(x)}\exp\sum_{k=1}^Kw_kf_k(y,x)\\
Z(x)&=\sum_y\exp\sum_{k=1}^Kw_kf_k(y,x)
\end{align}
$$
若以$w$表示权值向量， 即
$$
w=(w_1,w_2,\dots,w_K)^T
$$
以$F$表示全局特征向量，即
$$
F(y,x)=(f_1(y,x),f_2(y,x),\dots,f_K(y,x))^T
$$
条件随机场可以表示成向量内积的形式
$$
\begin{align}
P_w(y|x)&=\frac{\exp(w\cdot F(y,x))}{Z_w(x)}\\
Z_w(x)&=\sum_y\exp\left(w\cdot F(y,x)\right)
\end{align}
$$

### 矩阵形式
针对线性链条件随机场, 引入起点和终点状态标记$y_0=start,y_{n+1}=end$， 这时$P_w(y|x)$可以矩阵形式表示。

对应观测序列的**每个位置**$i=1,2,\dots,\color{red}{n+1}$，定义一个$m$阶矩阵（$m$是标记$y_i$取值的个数）
$$
\begin{align}
M_i(x)&=\left[M_i(y_{i-1},y_i|x)\right]\\
M_i(y_{i-1},y_i)&=\exp\left(W_i(y_{i-1},y_i|x)\right)\\
W_i(y_{i-1},y_i|x)&=\sum_{k=1}^Kw_kf_k(y_{i-1},y_i|x)
\end{align}
$$
把整个向量乘法按照**观测位置**拆成矩阵形式， 每个观测位置对应一个矩阵.

给定观测序列$x$，相应的标记序列$y$的非规范化概率可以通过该序列的$n+1$个矩阵适当元素的乘积$\prod_{i=1}^{n+1}M_i(y_{i-1},y_i|x)$表示。于是
$$
P_w(y|x)=\frac{1}{Z_w(x)}\prod_{i=1}^{n+1}M_i(y_{i-1},y_i|x)
$$
其中，$Z_w$为规范化因子，$\color{red}{是n+1个矩阵的乘积的(start,stop)元素}$：
$$
Z_w(x)=[M_1(x)M_2(x)\dots M_{n+1}(x))]_{start,stop}
$$

上面的式子展开一下，得到$n+1$个$\color{red}m$阶矩阵
$$
M_i(x)=\left[\exp\left(\sum_{k=1}^Kw_kf_k(y_{i-1},y_i|x)\right)\right], i=1,2,\dots,n+1
$$
这里面， 各个位置$(1,2,\dots,n+1)$的**随机矩阵**分别是
$$
\begin{align}
M_1(y_0,y_1|x)&=\exp\left(\sum_{k=1}^Kw_kf_k(y_0,y_1|x)\right)\nonumber\\
&=\exp (w_1f_1(y_0,y_1))\exp(w_2f_2(y_0,y_1))\dots\exp(w_Kf_K(y_0,y_1))\\
M_2(y_1,y_2|x)&=\exp\left(\sum_{k=1}^Kw_kf_k(y_1,y_2|x)\right)\nonumber\\
&=\exp (w_1f_1(y_1,y_2))\exp(w_2f_2(y_1,y_2))\dots\exp(w_Kf_K(y_1,y_2))\\
\cdots \\
M_{n+1}(y_n,y_{n+1}|x)&=\exp\left(\sum_{k=1}^Kw_kf_k(y_n,y_{n+1}|x)\right)\nonumber\\
&=\exp (w_1f_1(y_n,y_{n+1}))\exp(w_2f_2(y_n,y_{n+1}))\dots\exp(w_Kf_K(y_n,y_{n+1}))\\
\end{align}
$$

所以，无论特征有多少个，随机矩阵都是$(n+1)$个

In [24]:
#这里根据例11.2的启发整合为一个矩阵
F0 = S0
F1 = T1 + np.array(S1 * len(T1)).reshape(2, 2)
F2 = T2 + np.array(S2 * len(T2)).reshape(2, 2)
Y = [1, 2, 2]  #即书上例一需要计算的非规划条件概率的标记序列
Y = np.array(Y) - 1

P = np.exp(F0[Y[0]])
for i in range(1, len(Y)):
    PIter = np.exp((eval('F%d' % i)[Y[i - 1]][Y[i]]))
    P *= PIter
print('非规范化概率', P)

非规范化概率 24.532530197109345


###  例11.2
观测序列$x$，状态序列$y,i=1,2,3, n=3$，标记$y_i\in\{1,2\}$，假设$y_0=start=1,y_4=stop=1$，各个位置的随机矩阵为
$$
\begin{aligned}
M_1(x)=
\begin{bmatrix}
&a_{01}&a_{02}\\
&0&0
\end{bmatrix}
&,M_2(x)=
\begin{bmatrix}
&b_{11}&b_{12}\\
&b_{21}&b_{22}
\end{bmatrix}
\\
M_3(x)=
\begin{bmatrix}
&c_{11}&c_{12}\\
&c_{21}&c_{22}
\end{bmatrix}
&,M_4(x)=
\begin{bmatrix}
&1&0\\
&1&0
\end{bmatrix}
\end{aligned}
$$

由$M_i$的定义
$$
M_i(x)=\left[\exp\left(\sum_{k=1}^Kw_kf_k(y_{i-1},y_i|x)\right)\right], i=1,2,\dots,n+1\nonumber
$$
以及$y_i\in\{1,2\}$

可以知道，每个$M_i$中$f_k$对应的$y_{i-1},y_i$都有两种取值，对应的组合就有四种
$$
M(x)=
\begin{bmatrix}
\exp\sum\limits_{k=1}^Kw_kf_k(\color{red}{y[0],y[0]}),\exp\sum\limits_{k=1}^Kw_kf_k(\color{red}{y[0],y[1]})\\
\exp\sum\limits_{k=1}^Kw_kf_k(\color{red}{y[1],y[0]}),\exp\sum\limits_{k=1}^Kw_kf_k(\color{red}{y[1],y[1]})
\end{bmatrix}
$$
对应的红色部分组合使得$M$成为一个矩阵, $y[0], y[1]$指的是每个位置上的$Y$的具体取值(这里$y[0]=1, y[1]=2$)
$$
\begin{aligned}
&M_1\rightarrow y[0]\rightarrow start \\
&M_4\rightarrow y[4]\rightarrow end
\end{aligned}
$$

对于$y_0=start=1$
$$
M_1(x)=
\begin{bmatrix}
\exp\sum\limits_{k=1}^Kw_kf_k(y[0],y[0]),\exp\sum\limits_{k=1}^Kw_kf_k(y[0],y[1])\\
\exp\sum\limits_{k=1}^Kw_kf_k(\color{blue}{y[1]},y[0]),\exp\sum\limits_{k=1}^Kw_kf_k(\color{blue}{y[1]},y[1])
\end{bmatrix}
$$
蓝色表示该点上$y$取不到该值,所在元素的值为0. 同样的对于$y_4=stop=1$
$$
M_4(x)=
\begin{bmatrix}
\exp\sum\limits_{k=1}^Kw_kf_k(y[0],y[0]),\exp\sum\limits_{k=1}^Kw_kf_k(y[0],\color{blue}{y[1]})\\
\exp\sum\limits_{k=1}^Kw_kf_k(y[1],y[0]),\exp\sum\limits_{k=1}^Kw_kf_k(y[1],\color{blue}{y[1]})
\end{bmatrix}
$$


使用Symbol进行推导

In [12]:
from sympy import *

a01,a02, b11, b12, b21, b22, c11, c12, c21, c22  = symbols("a01, a02, \
                                                            b11, b12,b21, b22, \
                                                            c11, c12, c21, c22")
M1 = Matrix([[a01, a02],
             [0,   0]])
M2 = Matrix([[b11, b12],
             [b21, b22]])

M3 = Matrix([[c11, c12],
             [c21, c22]])

M4 = Matrix([[1, 0],
             [1, 0]])
Z = expand(M1*M2*M3*M4)  # 展开
Z

Matrix([
[a01*b11*c11 + a01*b11*c12 + a01*b12*c21 + a01*b12*c22 + a02*b21*c11 + a02*b21*c12 + a02*b22*c21 + a02*b22*c22, 0],
[                                                                                                            0, 0]])

In [8]:
P = str(expand(M1*M2*M3*M4)[0, 0]).replace(" ", "").split('+')
P

['a01*b11*c11',
 'a01*b11*c12',
 'a01*b12*c21',
 'a01*b12*c22',
 'a02*b21*c11',
 'a02*b21*c12',
 'a02*b22*c21',
 'a02*b22*c22']

In [15]:
# 体会各个路径之间关系
for i in range(2):
    for j in range(2):
        for k in range(2):
            print(f'i={i}, j={j}, k={k}', str(M1[0, i] * M2[i, j] * M3[j, k]))

i=0, j=0, k=0 a01*b11*c11
i=0, j=0, k=1 a01*b11*c12
i=0, j=1, k=0 a01*b12*c21
i=0, j=1, k=1 a01*b12*c22
i=1, j=0, k=0 a02*b21*c11
i=1, j=0, k=1 a02*b21*c12
i=1, j=1, k=0 a02*b22*c21
i=1, j=1, k=1 a02*b22*c22


## 概率计算问题
条件随机场地概率计算问题, 给定条件随机场$P(Y|X)$, 输入序列$x$和输出序列$y$, 计算条件概率$P(Y_i=y_i|x), \quad P(Y_{i-1}=y_{i-1}|x)$以及相应的数学期望.  
类似隐马尔可夫模型中的概率计算问题, 使用前向-后向算法. 
### 前向-后向算法
前向向量$\alpha_i(x)$
1. 初值
    $$\alpha_0(y|x)=
    \begin{cases}
    1,&y=start \\
    0,&others
    \end{cases}$$
2. 递推
   $$\alpha_i^T(y_i|x)=\alpha_{i-1}^T(y_{i-1}|x)[M_i(y_{i-1},y_i|x)],i=1,2,\dots,n+1$$

$\alpha_i(y_i|x)$表示在位置$i$的标记是$y_i$并且到位置$i$的前部标记序列的非规范化概率，$y_i$可取的值有$m$个，所以$\alpha_i(x)$是$m$维列向量

后向向量$\beta_i(x)$

1. 初值
   $$\beta_{n+1}(y_{n+1}|x)=
   \begin{cases}
   1,&y_{n+1}=stop \\
   0,&others
   \end{cases}$$
2. 递推
   $$\beta_i(y_i|x)=[M_{i+1}(y_i,y_{i+1}|x)]\beta_{i+1}(y_{i+1}|x),i=1,2,\dots,n+1$$

$\beta_i(y_i|x)$表示在位置$i$的标记是$y_i$并且从$i+1$到$n$的后部标记序列的非规范化概率
条件概率:
$$P(Y_i=y_i|x) = \frac {\alpha_i^T(y_i|x)\beta_i(y_i|x)}{Z(x)}$$
$$P(Y_{i-1}=y_{i-1}, Y_i=y_i|x) = \frac {\alpha_{i-1}^T(y_{i-1}|x)M_i(y_{i-1}, y_i|x)\beta_i(y_i|x)}{Z(x)}$$
$$
Z(x)=\alpha_n^T(x)\cdot \mathbf 1=\mathbf 1^T\cdot\beta_1(x)
$$
$\mathbf 1$是元素均为1的m维列向量

### 期望值的计算
利用前向-后向算法, 可以计算特征函数关于联合分布$P(X, Y)$和条件分布$P(Y|X)$的数学期望

特征函数$f_k$关于条件分布$P(Y|X)$的数学期望:
$$\begin{align}
E_{P(Y|X)}[f_k] &= \sum_y P(Y|X)f_k(y, x) \\
&= \sum_{i=1}^{n+1}\sum_{y_{i-1}, y_i}f_k(y_{i-1}, y_i, x, i)\frac {\alpha_{i-1}^T(y_{i-1}|x)M_i(y_{i-1}, y_i|x)\beta_i(y_i|x)}{Z(x)}
\end{align}$$
假设经验分布为$\tilde P(X)$特征函数$f_k$关于联合分布$P(Y, X)$的数学期望:
$$\begin{align}
E_{P(Y,X)}[f_k] &= \sum_{x,y} P(y, x)\sum_{i=1}^{n+1}f_k(y_{i-1}, y_i, x, i) \\
&= \sum_{x}\tilde P(x) \sum_y P(y|x)\sum_{i=1}^{n+1}f_k(y_{i-1}, y_i, x, i) \\
&= \sum_{x}\tilde P(x) \sum_{i=1}^{n+1}\sum_{y_{i-1}, y_i}f_k(y_{i-1}, y_i, x, i)\frac {\alpha_{i-1}^T(y_{i-1}|x)M_i(y_{i-1}, y_i|x)\beta_i(y_i|x)}{Z(x)}
\end{align}$$